<a href="https://colab.research.google.com/github/masonnlp/bioasq_qa_module/blob/master/Updated_QA_Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/My Drive/bert/ 
%pip install tensorflow-gpu==1.15
%pip install dicttoxml

/content/drive/My Drive/bert
     |████████████████████████████████| 411.5MB 22kB/s 
     |████████████████████████████████| 3.8MB 34.9MB/s 
     |████████████████████████████████| 512kB 33.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=721179350763180010998875d7b1f83f21c1f26d0c912b39312c26bf2be2c799
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.3.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 

In [ ]:
from bs4 import BeautifulSoup as bs
import json
from json import loads
from dicttoxml import dicttoxml
import os
content = []
paragraphs=[]
paragraphs1=[]
data ={}

# Read the XML file and preprocess input
with open("/content/demo.xml", "r") as file:
    # Read each line in the file, readlines() returns a list of lines
    content = file.readlines()
    # Combine the lines in the list into a string
    data= {}
    data1={}
    content = "".join(content)
    bs_content = bs(content, "lxml")
    result =(bs_content.find_all("q"))
    #print(result)
    for item in result:
      q_type = (item.find('qp').find('type').get_text())
      #print(q_type)
      if q_type == 'Factoid' or q_type == 'List':
         id= (item.attrs['id'])
         #print(id)
         q = (item.text.split('?' or '.')[0])
         context = (item.find('ir').find('result').find('abstract').get_text())
         qa = [{'id': id, "question": q}]
         dict1={'qas':qa,'context':context}
         #print(dict1)
         paragraphs.append(dict1)  
         #print(paragraphs)
         data['data'] = [{"paragraphs": paragraphs}]
         #print(data)
         with open('/content/file_factoid_new.json', 'w') as outfile:
          json.dump(data, outfile, indent=4)
          outfile.close()
      elif q_type == 'Summary':
        pass
            
      elif q_type == 'Yesno':
         id1= (item.attrs['id'])
         q1 = (item.text.split('?' or '.')[0])
         context1 = (item.find('ir').find('result').find('abstract').get_text())
         qa1 = [{'id': id1, "question": q1}]
         dict2={'qas':qa1,'context':context1}
         paragraphs1.append(dict2)  
         data1['data'] = [{"paragraphs": paragraphs1}]
     
         with open('/content/file_yesno.json', 'w') as outfile:
          json.dump(data1, outfile, indent=4)
          outfile.close()
os.system("python run_squad.py \
          --vocab_file=/content/vocab.txt \
          --bert_config_file=/content/bert_config.json \
          --init_checkpoint=../BERT-pubmed-1000000-SQuAD/model.ckpt-14599 \
          --do_train=False \
          --max_query_length=30  \
          --do_predict=True \
          --predict_file=/content/file_factoid.json \
          --predict_batch_size=8 \
          --n_best_size=3 \
          --max_seq_length=384 \
          --doc_stride=128 \
          --output_dir=/content/hi/")
json_file_path = "/content/hi/predictions.json"
with open(json_file_path, 'r') as j:
     contents = json.loads(j.read())
xml = dicttoxml(contents)

with open('/content/bert_predictions.xml', 'wb') as outfile:
    outfile.write(xml)
    outfile.close()


os.system("python ../bioasq-biobert-1.0/run_yesno.py \
  --do_train=True \
	--do_predict=True \
	--vocab_file=/content/vocab.txt \
	--bert_config_file=/content/bert_config.json \
	--init_checkpoint=./BERT-pubmed-1000000-SQuAD2/model.ckpt-14470 \
	--max_seq_length=384 \
	--train_batch_size=10 \
	--learning_rate=5e-6 \
	--doc_stride=128 \
	--do_lower_case=False \
	--num_train_epochs=2 \
	--train_file=/content/BioASQ-train-yesno-7b-snippet.json \
	--predict_file=/content/file_yesno.json \
	--output_dir=/content/output_yesno")

json_file_path = "/content/output_yesno/predictions.json"
with open(json_file_path, 'r') as j:
     contents = json.loads(j.read())
xml = dicttoxml(contents)

with open('/content/bert_predictions.xml', 'ab') as outfile:
    outfile.write(xml)
    outfile.close()
   